# Group Details

## Group Name: Group 08

### Student 1: Jasper Linders

### Student 2: Alexander Liu

### Student 3: Sjoerd van Straten

# Loading Data and Preliminaries

In [60]:
# from google.colab import drive
# drive.mount('/content/drive')

In [61]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [63]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")

In [64]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""
example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [65]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time.
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.
"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-5.85725792 -5.394571           nan         nan]
 [-6.03781257 -5.72445953         nan         nan]
 [-0.90623054 -6.93416278         nan         nan]
 [ 2.83149339 -7.50100819         nan         nan]
 [-2.85586881  1.77667501         nan         nan]
 [ 4.04424526  4.00563603         nan         nan]
 [-5.24887713 -4.83081005         nan         nan]
 [-5.81391023 -5.1109838          nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


# Data Handling and Preprocessing

First, let's import the data and make variables for the train and test data.

In [66]:
import os
from torch.utils.data import Dataset, DataLoader, TensorDataset

class ImportData(Dataset):
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.file_list = sorted(os.listdir(folder_path))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        file_name = self.file_list[index]
        file_path = os.path.join(self.folder_path, file_name)
        data, label = load_array(file_path, task='task 2')
        return data, label

# Create an instance of the custom dataset class with the folder path
train_import = ImportData('data/task 2_3/train/')
test_import = ImportData('data/task 2_3/test/')

X_train_import = []
y_train_import = []
X_test_import = []
y_test_import = []

# Iterate through the train_dataset to extract data and labels
for data, label in train_import:
    X_train_import.append(data)
    y_train_import.append(label)

for data, label in test_import:
    X_test_import.append(data)
    y_test_import.append(label)

max_length = 9
max_seq = 49

# Pad the data samples with zeros to have the same shape
X_train_padded = []
for data in X_train_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_train_padded.append(padded_data)

y_train_padded = []
for label in y_train_import:
    pad_width = max_length - label.shape[0]
    pad_length = max_seq - label.shape[1]
    padded_label = np.pad(label, ((0, pad_width), (0, pad_length), (0,0)), mode='constant')
    y_train_padded.append(padded_label)

X_test_padded = []
for data in X_test_import:
    pad_width = max_length - data.shape[0]
    padded_data = np.pad(data, ((0, pad_width), (0, 0)), mode='constant')
    X_test_padded.append(padded_data)

y_test_padded = []
for label in y_test_import:
    pad_width = max_length - label.shape[0]
    pad_length = max_seq - label.shape[1]
    padded_label = np.pad(label, ((0, pad_width), (0, pad_length), (0,0)), mode='constant')
    y_test_padded.append(padded_label)

# Convert the padded data and labels to tensors
X_train = torch.tensor(np.array(X_train_padded))
y_train = torch.tensor(np.array(y_train_padded))
X_test = torch.tensor(np.array(X_test_padded))
y_test = torch.tensor(np.array(y_test_padded))

# Print the shape of X_train and the first label in y_train
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  torch.Size([900, 9, 5])
y_train shape:  torch.Size([900, 9, 49, 4])
X_test shape:  torch.Size([100, 9, 5])
y_test shape:  torch.Size([100, 9, 49, 4])


# Model Implementation

In [99]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Training on {torch.cuda.get_device_name(0)}' if torch.cuda.is_available() else 'Training on CPU')

Training on Quadro M1200


In [67]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=9*5, hidden_size=128, num_layers=49, proj_size=36, batch_first=True)
        # self.linear = nn.Linear(128, 45)

    def forward(self, x):
        output, (h_n, c_n) = self.lstm(x)
        # output, (h_n, c_n) = self.linear(output), self.linear(h_n), self.linear(c_n)
        return output, (h_n, c_n)

In [68]:
X_train[0].flatten().unsqueeze(0).float().shape

torch.Size([1, 45])

In [69]:
y_train[0].shape

torch.Size([9, 49, 4])

In [70]:
y_train[0]

tensor([[[-5.3508,  4.7698, -0.6015, -2.7740],
         [-5.3998,  4.5065, -0.3956, -2.5268],
         [-5.4322,  4.2605, -0.2609, -2.4050],
         ...,
         [-0.5797, -4.2417,  1.7654, -4.3593],
         [-0.3841, -4.6990,  2.1710, -4.7872],
         [-0.1408, -5.1981,  2.7102, -5.1801]],

        [[ 4.2678, -0.3754, -0.6071, -2.9297],
         [ 4.2057, -0.6669, -0.6349, -2.8997],
         [ 4.1408, -0.9554, -0.6621, -2.8695],
         ...,
         [-0.6855, -9.6195, -1.2635, -0.4569],
         [-0.8106, -9.6572, -1.2388, -0.2947],
         [-0.9331, -9.6777, -1.2110, -0.1126]],

        [[-4.9651,  5.3952,  1.6454, -0.1498],
         [-4.8168,  5.3516,  1.3514, -0.6671],
         [-4.6911,  5.2681,  1.1729, -0.9826],
         ...,
         [-0.7739, -6.5166,  0.6282, -2.4154],
         [-0.6906, -6.7618,  1.0373, -2.5269],
         [-0.5715, -7.0326,  1.2732, -2.9555]],

        ...,

        [[-0.6005, 16.5437, -0.9493, 14.9669],
         [-0.6955, 18.0398, -0.9503, 14.9556]

In [71]:
test = y_train[0].permute(1, 0, 2).contiguous().view(49, -1)
test

tensor([[-5.3508,  4.7698, -0.6015,  ...,  0.0000,  0.0000,  0.0000],
        [-5.3998,  4.5065, -0.3956,  ...,  0.0000,  0.0000,  0.0000],
        [-5.4322,  4.2605, -0.2609,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.5797, -4.2417,  1.7654,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3841, -4.6990,  2.1710,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1408, -5.1981,  2.7102,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [72]:
y_train[0].permute(1, 0, 2).contiguous().view(49, -1).shape

torch.Size([49, 36])

In [73]:
model = LSTM()
output, (h_n, c_n) = model(X_train[0].flatten().unsqueeze(0).float())
output.shape, c_n.shape, h_n.shape

(torch.Size([1, 36]), torch.Size([49, 128]), torch.Size([49, 36]))

# Model Training

In [122]:
model = LSTM()
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Set the number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Iterate over the instances of the training dataset
    for input, label in zip(X_train, y_train):
        # print(input.shape, label.shape)
        input = input.flatten().unsqueeze(0).float()
        label = label.permute(1, 0, 2).contiguous().view(49, -1).float()
        input.to(device), label.to(device)
        # print(input.shape, label.shape)
        optimizer.zero_grad()
        output, (h_n, c_n) = model(input)
        # print(output.shape, c_n.shape, h_n.shape)
        loss = criterion(h_n, label)
        # loss = loss.float()
        # print(loss)
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {loss.item()}")

Epoch 1/10, Training Loss: 1.1881932020187378
Epoch 2/10, Training Loss: 1.1564948558807373
Epoch 3/10, Training Loss: 1.1183125972747803
Epoch 4/10, Training Loss: 1.1372084617614746
Epoch 5/10, Training Loss: 1.1003795862197876
Epoch 6/10, Training Loss: 1.075750470161438
Epoch 7/10, Training Loss: 1.0650378465652466
Epoch 8/10, Training Loss: 1.1167699098587036
Epoch 9/10, Training Loss: 1.0648481845855713
Epoch 10/10, Training Loss: 1.0733636617660522


In [121]:
idx = 500

input = X_train[idx].flatten().unsqueeze(0).float()
label = y_train[idx].permute(1, 0, 2).contiguous().view(49, -1).float()
output, (h_n, c_n) = model(input)
loss = criterion(h_n, label)
loss

tensor(2.8600, grad_fn=<MeanBackward0>)

In [117]:
idx = 1

input = X_test[idx].flatten().unsqueeze(0).float()
label = y_test[idx].permute(1, 0, 2).contiguous().view(49, -1).float()
output, (h_n, c_n) = model(input)
loss = criterion(h_n, label)
loss

tensor(nan, grad_fn=<MeanBackward0>)